In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate

import halotools

## Chris code

In [ ]:
# Given the b_params for the behroozi functional form, and the halos in the sim
# find the SM for each halo
def get_sm_for_sim(sim_data, b_params, s_params, x_field, sanity=False):
    assert len(b_params) == 5 and len(s_params) == 2

    log_halo_masses = np.log10(sim_data[x_field])
    min_mvir = np.min(log_halo_masses)
    max_mvir = np.max(log_halo_masses)

    sample_halo_masses = np.linspace(min_mvir, max_mvir, num=12)

    try:
        sample_stellar_masses = smhm_fit.f_shmr(
            sample_halo_masses,
            10**b_params[0],
            10**b_params[1],
            *b_params[2:],
        )
    except Exception as e:
        if e.args[0].startswith("Failure to invert"):
            return np.zeros_like(log_halo_masses)
        raise


    f_mvir_to_sm = scipy.interpolate.interp1d(sample_halo_masses, sample_stellar_masses)

    log_stellar_masses = f_mvir_to_sm(log_halo_masses)
    if not np.all(np.isfinite(log_stellar_masses)):
        print("infinite SM")
        return np.zeros_like(log_stellar_masses)

    # This adds some stochasticity... Ideally we would keep these as a distribution
    # But that is much harder. So we just accept the stochasticity and that the MCMC
    # will take longer to converge

    log_sm_scatter = s_params[0] * log_halo_masses + s_params[1]
    if not np.all(log_sm_scatter > 0):
        print("negative scatter")
        return np.zeros_like(log_stellar_masses)

    log_stellar_masses += np.random.normal(0, log_sm_scatter, size=len(log_sm_scatter))

    if sanity:
        return log_stellar_masses, sample_halo_masses, sample_stellar_masses, f_mvir_to_sm, min_mvir, max_mvir
    else:
        return log_stellar_masses

In [ ]:
mass_b_params = [12.52, 10.91, 0.45, 0.6, 1.83]
mass_s_params = [-0.06, 1.08]
mass_x_field = "halo_mvir"

doesnt seem like chris' code implements using Vmax@Mpeak to calculate stellar mass...

Check his notebooks